## Migrating to tensor regression

The EVs are unexpectedly bad across different neural networks. This could either be real difference in the expressed function spaces but it's also likely that we are mis-specifying the source-target model. One obvious issue is that vectorization below fully connected layers destroys a lot of interal structure in the tensor output of convolution layers. Moreoever, vectorization of tensors of even moderate dimensions exponentially inflate the number of predictive variables. For these reasons we should consider low-rank or regularized tensor regression. Here is an evaluation of tensorly.

In [2]:
from tensorly.base import tensor_to_vec, partial_tensor_to_vec
from tensorly.random import check_random_state
from tensorly.regression.cp_regression import CPRegressor
from tensorly.regression.tucker_regression import TuckerRegressor
import tensorly as tl
from tqdm import tqdm

import torch
import random
import torchvision.models as models
from util.modelregressions import CNNCrossFit
from util.misc_functions import float2rgb
from dataset.hvm import HVMDataset
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
import os

tl.set_backend('pytorch')
device = 'cuda'

In [3]:
# load up target and source models
squeezenet_target = models.squeezenet1_1(pretrained=True).features
squeezenet_source = models.squeezenet1_0(pretrained=True).features
# define basic params 
target_layer = 12
source_layer = 12
source_channels = np.arange(512)
target_channels = np.arange(100)
# pre-load data into memory for speed
n_data = 5000;
n_train = 4000;
n_test = 1000;
hvmdataset = HVMDataset('cpu',n_data)
# create a control experiment (squeezenet1_1-->squeezenet1_0)
snet2snet = CNNCrossFit(squeezenet_target,squeezenet_source,target_layer,source_layer,target_channels,source_channels,device)
# extract layer activation across two models
snet2snet.design(hvmdataset, batch_size = 10, shuffle = False)
# train test split
snet2snet.train_test_split(n_train,n_test)

loading and preprocessing hvm


100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:39<00:00, 126.63it/s]


Extracting activations


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:18<00:00, 26.91it/s]


In [3]:
# extract X and Y tensor
X_train = tl.tensor(snet2snet.sor_train,device=device)
X_test = tl.tensor(snet2snet.sor_test,device=device)

In [4]:
import random
n_units = 200
rand_set = random.sample(range(0,len(snet2snet.indexer)),len(snet2snet.indexer))
valididx = np.where((snet2snet.indexer.get_tar(rand_set)
rand_units = [rand_set[u] for i,u in enumerate(valididx[:n_units])]
n_units = len(rand_units)

TypeError: '>' not supported between instances of 'list' and 'int'

We will compare rank-1 Tensor Regression to Orthogonal Matching Pursuit because that had the second best EV score in Jonas's original experiments on Ko's data and doesn't need much tuning. 

In [ ]:
from sklearn.metrics import explained_variance_score as EV
from sklearn.linear_model import LinearRegression, BayesianRidge, OrthogonalMatchingPursuit

ncs = [1,2,3,4,5,6,7,8]
n_ncs = len(ncs)

rw = np.zeros((n_ncs, n_units,snet2snet.n_sor,15,15))
X_train_LR = snet2snet.sor_train.reshape(n_train,-1)
X_test_LR = snet2snet.sor_test.reshape(n_test,-1)
Y_pred_test_LR = np.zeros((n_test,n_units,n_ncs))
Y_pred_train_LR = np.zeros((n_train,n_units,n_ncs))


test_score_LR = np.zeros((n_units,n_ncs))
train_score_LR = np.zeros((n_units,n_ncs))
for i,u in enumerate(tqdm(rand_units)):
    index = snet2snet.indexer.get_sur(u)
    Y_train_LR = snet2snet.tar_train[:,index[0],index[1],index[2]]
    Y_test_LR = snet2snet.tar_test[:,index[0],index[1],index[2]]
    for j, nc in enumerate(ncs):
        estimator_LR = OrthogonalMatchingPursuit(n_nonzero_coefs=nc)
        #estimator_LR = BayesianRidge()
        estimator_LR.fit(X_train_LR, Y_train_LR)
        rw[j,i] = estimator_LR.coef_.reshape(snet2snet.n_sor,15,15)
        # score on train and test set
        Y_pred_test_LR[:,i,j] = estimator_LR.predict(X_test_LR)
        Y_pred_train_LR[:,i,j] = estimator_LR.predict(X_train_LR)
        test_score_LR[i,j] = EV(Y_test_LR,Y_pred_test_LR[:,i,j])
        train_score_LR[i,j] = EV(Y_train_LR,Y_pred_train_LR[:,i,j])

In [ ]:
plt.plot(np.median(train_score_LR,axis=0))
plt.plot(np.median(test_score_LR,axis=0))
plt.show()

In [ ]:
tw = np.zeros((n_units,snet2snet.n_sor,15,15))
ttw = []
test_score = np.zeros(n_units)
train_score = np.zeros(n_units)
Y_pred_test = tl.tensor(np.zeros((n_test,n_units)),device=device)
Y_pred_train = tl.tensor(np.zeros((n_train,n_units)),device=device)
Y_test = tl.tensor(np.zeros((n_test,n_units)),device=device)
Y_train = tl.tensor(np.zeros((n_train,n_units)),device=device)
for i,u in enumerate(tqdm(rand_units)):
    index = snet2snet.indexer.get_sur(u)
    Y_train[:,i] = tl.tensor(snet2snet.tar_train[:,index[0],index[1],index[2]],device=device)
    Y_test[:,i] = tl.tensor(snet2snet.tar_test[:,index[0],index[1],index[2]],device=device)
    # Create a Tucker Regressor estimator
    estimator = TuckerRegressor(weight_ranks=[1,1,1], tol=1e-5, n_iter_max=50, reg_W=10, verbose=0)
    # Fit the estimator to the dat
    estimator.fit(X_train, Y_train[:,i])
    # score on train and test set
    Y_pred_test[:,i] = estimator.predict(X_test)
    Y_pred_train[:,i] = estimator.predict(X_train)
    tw[i,:,:,:] = estimator.weight_tensor_.cpu().detach().squeeze().numpy()
    ttw.append(estimator.tucker_weight_)
    test_score[i] = EV(Y_test[:,i].cpu().detach().numpy(),Y_pred_test[:,i].cpu().detach().numpy())
    train_score[i] = EV(Y_train[:,i].cpu().detach().numpy(),Y_pred_train[:,i].cpu().detach().numpy())

In [ ]:
comp = 4

i = np.where(test_score>.25)[0][0]
idx = snet2snet.indexer.get_sur(rand_units[i])

plt.figure(figsize = (10,6))
ax0 = plt.subplot(2,3,1)
plt.plot(Y_train_LR,Y_pred_train_LR[:,i,comp],'.')
plt.plot(Y_test_LR,Y_pred_test_LR[:,i,comp],'.')
plt.plot([-100,100],[-100,100],'k--')
plt.xlim([0,10])
plt.ylim([0,10])
ax0.set_aspect('equal')
plt.xlabel('target activation')
plt.ylabel('OMP predicted activation')

ax1 = plt.subplot(2,3,4)
plt.plot(Y_train[:,i].cpu(),Y_pred_train[:,i].cpu(),'.')
plt.plot(Y_test[:,i].cpu(),Y_pred_test[:,i].cpu(),'.')
plt.plot([-100,100],[-100,100],'k--')
plt.xlim([0,10])
plt.ylim([0,10])
ax1.set_aspect('equal')
plt.xlabel('target activation')
plt.ylabel('Tensor predicted activation')

ax2 = plt.subplot(2,3,2)
plt.imshow(np.sum(rw[comp,i],axis=0).T)
plt.plot(idx[1],idx[2],'rx')
ax2.get_xaxis().set_ticks([])
ax2.axes.get_yaxis().set_ticks([])
plt.xlabel('w')
plt.ylabel('h')

ax3 = plt.subplot(2,3,5)
plt.imshow((ttw[i][1][1].cpu().T*ttw[i][1][2].cpu())**2)
plt.plot(idx[1],idx[2],'rx')
ax3.get_xaxis().set_ticks([])
ax3.axes.get_yaxis().set_ticks([])
plt.xlabel('w')
plt.ylabel('h')

ax4 = plt.subplot(2,3,3)
plt.plot(np.sum(np.sum(rw[comp,i],axis=1),axis=1))
ax4.get_xaxis().set_ticks([])
ax4.axes.get_yaxis().set_ticks([])
plt.xlim([0,snet2snet.n_sor])
plt.ylabel('weight')

ax5 = plt.subplot(2,3,6)
plt.plot(ttw[i][1][0].cpu())
ax5.get_xaxis().set_ticks([])
ax5.axes.get_yaxis().set_ticks([])
plt.xlim([0,snet2snet.n_sor])
plt.ylabel('weight')
plt.xlabel('feature')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
ax1 = plt.subplot(1,2,1)
ax1.set_aspect('equal')
plt.scatter(train_score_LR, test_score_LR, s=10, label='OMP')
plt.scatter(train_score, test_score, s=10, label='Tensor')
plt.plot([-1,1],[-1,1],'k--')
plt.xlim([-1,1])
plt.ylim([-1,1])
plt.xlabel(r'${f_{source}(x),f_{target}(x)}$  train EV', fontsize=10)
plt.ylabel(r'${f_{source}(x),f_{target}(x)}$  test EV', fontsize=10)
plt.legend()
ax1 = plt.subplot(1,2,2)
ax1.set_aspect('equal')
plt.plot([-1,1],[-1,1],'k--')
plt.scatter(test_score_LR, test_score,s=10)
plt.xlim([-1,1])
plt.ylim([-1,1])
plt.xlabel(r'OMP test EV', fontsize=10)
plt.ylabel(r'Tensor  test EV', fontsize=10)
plt.show()

In [ ]:
idx = snet2snet.indexer.get_sur(rand_units[499])
plt.hist(snet2snet.Xtar[:,idx[0],idx[1],idx[2]],100)
plt.yscale('log', nonposy='clip')
plt.show()

In [ ]:
plt.hist(test_score,bins=np.arange(-1,1,.05))
plt.show()

In [ ]:
test_score_LR[-1]